# import modules & load files

import modules

In [24]:
import pandas as pd
import numpy as np
import math
import glob
import datetime
import os
current_directory = os.getcwd()
print("현재 디렉토리:", current_directory)

현재 디렉토리: d:\myhub\Project1


load files

In [25]:
# strike price set
CTB9sp = pd.read_csv('./target_price_set/CTB9sp.csv', index_col=0)
CTB15sp = pd.read_csv('./target_price_set/CTB15sp.csv', index_col=0)
PTB9sp = pd.read_csv('./target_price_set/PTB9sp.csv', index_col=0)
PTB15sp = pd.read_csv('./target_price_set/PTB15sp.csv', index_col=0)

In [26]:
# predict labels
output = pd.read_csv("./result_quarter_output.csv", index_col=0)
labels = output.iloc[:,-7:]
labels[['cost', 'revenue']] = 0

In [27]:
labels

,VKOSPI_Label,LogisticRegression,DecisionTree,SVC,RandomForest,XGB,LGBM,cost,revenue
2023-10-10,1.0,0.0,0.0,0.0,0.0,0,0.0,0,0
2023-10-11,1.0,0.0,0.0,0.0,0.0,0,0.0,0,0
2023-10-12,1.0,0.0,0.0,0.0,0.0,0,0.0,0,0
2023-10-13,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0
2023-10-16,0.0,0.0,0.0,0.0,0.0,1,1.0,0,0
2023-10-17,0.0,1.0,1.0,1.0,1.0,0,0.0,0,0
2023-10-18,1.0,0.0,0.0,0.0,0.0,0,0.0,0,0
2023-10-19,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0
2023-10-20,0.0,1.0,1.0,1.0,1.0,1,1.0,0,0
2023-10-23,1.0,1.0,0.0,1.0,1.0,1,1.0,0,0


In [28]:
labels.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19 entries, 2023-10-10 to 2023-11-03
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   VKOSPI_Label        19 non-null     float64
 1   LogisticRegression  19 non-null     float64
 2   DecisionTree        19 non-null     float64
 3   SVC                 19 non-null     float64
 4   RandomForest        19 non-null     float64
 5   XGB                 19 non-null     int64  
 6   LGBM                19 non-null     float64
 7   cost                19 non-null     int64  
 8   revenue             19 non-null     int64  
dtypes: float64(6), int64(3)
memory usage: 1.5+ KB


In [29]:
labels.describe()

,VKOSPI_Label,LogisticRegression,DecisionTree,SVC,RandomForest,XGB,LGBM,cost,revenue
count,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.0,19.0
mean,0.526316,0.263158,0.368421,0.210526,0.263158,0.315789,0.263158,0.0,0.0
std,0.512989,0.452414,0.495595,0.418854,0.452414,0.477567,0.452414,0.0,0.0
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
50%,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
75%,1.000000,0.500000,1.000000,0.000000,0.500000,1.000000,0.500000,0.0,0.0
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,0.0


In [30]:
VKOSPI_Label = labels[['VKOSPI_Label', 'cost', 'revenue']]
LogisticRegression = labels[['LogisticRegression', 'cost', 'revenue']]
DecisionTree = labels[['DecisionTree', 'cost', 'revenue']]
SVC = labels[['SVC', 'cost', 'revenue']]
RandomForest = labels[['RandomForest', 'cost', 'revenue']]
XGB = labels[['XGB', 'cost', 'revenue']]
LGBM = labels[['LGBM', 'cost', 'revenue']]

In [31]:
VKOSPI_Label.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19 entries, 2023-10-10 to 2023-11-03
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   VKOSPI_Label  19 non-null     float64
 1   cost          19 non-null     int64  
 2   revenue       19 non-null     int64  
dtypes: float64(1), int64(2)
memory usage: 608.0+ bytes


매매전략별 수익률 계산
=====
if label == -1: # 하락 → 양매도
    인덱스(시간) 일치
    case 1: 스트래들매도
    CTB9sp에서 인덱스(시간) 기준행사가 컬럼 값을 저장 매도함
    PTB9sp에서 인덱스(시간) 기준행사가 컬럼 값을 저장 매도함
    CTB15sp에서 인덱스(시간) 기준행사가 컬럼 값을 저장 매도(청산)함
    PTB15sp에서 인덱스(시간) 기준행사가 컬럼 값을 저장 매수(청산)함
    
    case 2: 스트랭글
    CTB9sp에서 인덱스(시간) 기준행사가+4 C매도
    PTB9sp에서 인덱스(시간) 기준행사가-4 P매도
    CTB15sp에서 인덱스(시간) 기준행사가+4 C매수
    PTB15sp에서 인덱스(시간) 기준행사가-4 P매수
    수익률 계산 = 매수 값 / 매도 값
    수익금 계산 = 매수 값 - 매도 값

    case 3: 버터플매수
        case 3-1:
            기준행사가 P매도 *2
            기준행사가-1 P매수 
            기준행사가+1 P매수
        case 3-2:
            기준행사가 C매도 *2
            기준행사가-1 C매수
            기준행사가+1 C매수
    
elif Label == 0: # 보합
    관망 how to code
    
elif label == 1: # 상승 → 양매수
    인덱스(시간) 일치
    case 1: 스트래들매수
    CTB9sp에서 인덱스(시간) 기준행사가 값을 저장 매수함
    PTB9sp에서 인덱스(시간) 기준행사가 값을 저장 매수함
    CTB15sp에서 인덱스(시간) 기준행사가 값을 저장 매도(청산)함
    PTB15sp에서 인덱스(시간) 기준행사가 값을 저장 매도(청산)함
    수익률 계산 = 매도 값 / 매수 값
    수익금 계산 = 매도 값 - 매수 값

    case 2: 스트랭글매수
    CTB9sp에서 인덱스(시간) 기준행사가+4 C매수
    PTB9sp에서 인덱스(시간) 기준행사가-4 P매수
    CTB15sp에서 인덱스(시간) 기준행사가+4 C매도
    PTB15sp에서 인덱스(시간) 기준행사가-4 P매도

    case 3: 버터플매수
        case 3-1:
            기준행사가 P매수 *2
            기준행사가-1 P매도
            기준행사가+1 P매도
        case 3-2:
            기준행사가 C매수 *2
            기준행사가-1 C매도
            기준행사가+1 C매도

실제 변동

In [32]:
for i in range(len(VKOSPI_Label.index)):
    d = VKOSPI_Label.index[i]
    if VKOSPI_Label.loc[d, 'VKOSPI_Label'] == 0: # 변동성 하락 양매도 스트랭글
        try:
            cost = 0
            cost += CTB9sp['S+4'].loc[d]
            cost += PTB9sp['S-4'].loc[d]
            VKOSPI_Label['cost'].loc[d] = cost
            revenue = 0
            revenue += CTB15sp['S+4'].loc[d]
            revenue += CTB9sp['S-4'].loc[d]
            VKOSPI_Label['revenue'].loc[d] = revenue
        except:
            VKOSPI_Label['cost'].loc[d] = np.nan
            VKOSPI_Label['revenue'].loc[d] = np.nan
    else: # label == 1 변동성 상승 양매수 스트랭글
        try:
            cost = 0
            cost += CTB9sp['기준행사가'].loc[d]
            cost += PTB9sp['기준행사가'].loc[d]
            VKOSPI_Label['cost'].loc[d] = cost
            revenue = 0
            revenue += CTB15sp['기준행사가'].loc[d]
            revenue += CTB9sp['기준행사가'].loc[d]
            VKOSPI_Label['revenue'].loc[d] = revenue
        except:
            VKOSPI_Label['cost'].loc[d] = np.nan
            VKOSPI_Label['revenue'].loc[d] = np.nan

C:\Users\jay_hook\AppData\Local\Temp\ipykernel_6808\2437068338.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  VKOSPI_Label['cost'].loc[d] = cost
C:\Users\jay_hook\AppData\Local\Temp\ipykernel_6808\2437068338.py:21: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '13.74' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  VKOSPI_Label['cost'].loc[d] = cost
C:\Users\jay_hook\AppData\Local\Temp\ipykernel_6808\2437068338.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  VKOSPI_Label['revenue'].loc[d] = revenue

In [33]:
VKOSPI_Label['profit'] = VKOSPI_Label['revenue'] - VKOSPI_Label['cost']
VKOSPI_Label

C:\Users\jay_hook\AppData\Local\Temp\ipykernel_6808\1588952584.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  VKOSPI_Label['profit'] = VKOSPI_Label['revenue'] - VKOSPI_Label['cost']


,VKOSPI_Label,cost,revenue,profit
2023-10-10,1.0,13.74,18.43,4.69
2023-10-11,1.0,14.00,NaN,NaN
2023-10-12,1.0,12.52,19.14,6.62
2023-10-13,0.0,3.72,13.16,9.44
2023-10-16,0.0,3.59,13.55,9.96
2023-10-17,0.0,3.55,18.12,14.57
2023-10-18,1.0,10.34,14.27,3.93
2023-10-19,0.0,3.48,9.84,6.36
2023-10-20,0.0,4.62,9.78,5.16
2023-10-23,1.0,NaN,NaN,NaN


LogisticRegression

In [34]:
for i in range(len(LogisticRegression.index)):
    d = LogisticRegression.index[i]
    if LogisticRegression.loc[d, 'LogisticRegression'] == 0: # 변동성 하락 양매도 스트랭글
        try:
            cost = 0
            cost += CTB9sp['S+4'].loc[d]
            cost += PTB9sp['S-4'].loc[d]
            LogisticRegression['cost'].loc[d] = cost
            revenue = 0
            revenue += CTB15sp['S+4'].loc[d]
            revenue += CTB9sp['S-4'].loc[d]
            LogisticRegression['revenue'].loc[d] = revenue
        except:
            LogisticRegression['cost'].loc[d] = np.nan
            LogisticRegression['revenue'].loc[d] = np.nan
    else: # label == 1 변동성 상승 양매수 스트랭글
        try:
            cost = 0
            cost += CTB9sp['기준행사가'].loc[d]
            cost += PTB9sp['기준행사가'].loc[d]
            LogisticRegression['cost'].loc[d] = cost
            revenue = 0
            revenue += CTB15sp['기준행사가'].loc[d]
            revenue += CTB9sp['기준행사가'].loc[d]
            LogisticRegression['revenue'].loc[d] = revenue
        except:
            LogisticRegression['cost'].loc[d] = np.nan
            LogisticRegression['revenue'].loc[d] = np.nan
LogisticRegression['profit'] = LogisticRegression['revenue'] - LogisticRegression['cost']
LogisticRegression

C:\Users\jay_hook\AppData\Local\Temp\ipykernel_6808\1934238159.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  LogisticRegression['cost'].loc[d] = cost
C:\Users\jay_hook\AppData\Local\Temp\ipykernel_6808\1934238159.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '5.89' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  LogisticRegression['cost'].loc[d] = cost
C:\Users\jay_hook\AppData\Local\Temp\ipykernel_6808\1934238159.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  LogisticRegression['revenue'].l

,LogisticRegression,cost,revenue,profit
2023-10-10,0.0,5.89,NaN,NaN
2023-10-11,0.0,6.12,NaN,NaN
2023-10-12,0.0,4.92,20.03,15.11
2023-10-13,0.0,3.72,13.16,9.44
2023-10-16,0.0,3.59,13.55,9.96
2023-10-17,1.0,10.81,15.48,4.67
2023-10-18,0.0,3.25,16.32,13.07
2023-10-19,0.0,3.48,9.84,6.36
2023-10-20,1.0,11.71,6.79,-4.92
2023-10-23,1.0,NaN,NaN,NaN


DecisionTree

In [35]:
for i in range(len(DecisionTree.index)):
    d = DecisionTree.index[i]
    if DecisionTree.loc[d, 'DecisionTree'] == 0: # 변동성 하락 양매도 스트랭글
        try:
            cost = 0
            cost += CTB9sp['S+4'].loc[d]
            cost += PTB9sp['S-4'].loc[d]
            DecisionTree['cost'].loc[d] = cost
            revenue = 0
            revenue += CTB15sp['S+4'].loc[d]
            revenue += CTB9sp['S-4'].loc[d]
            DecisionTree['revenue'].loc[d] = revenue
        except:
            DecisionTree['cost'].loc[d] = np.nan
            DecisionTree['revenue'].loc[d] = np.nan
    else: # label == 1 변동성 상승 양매수 스트랭글
        try:
            cost = 0
            cost += CTB9sp['기준행사가'].loc[d]
            cost += PTB9sp['기준행사가'].loc[d]
            DecisionTree['cost'].loc[d] = cost
            revenue = 0
            revenue += CTB15sp['기준행사가'].loc[d]
            revenue += CTB9sp['기준행사가'].loc[d]
            DecisionTree['revenue'].loc[d] = revenue
            print(DecisionTree['revenue'].loc[d])
        except:
            DecisionTree['cost'].loc[d] = np.nan
            DecisionTree['revenue'].loc[d] = np.nan

C:\Users\jay_hook\AppData\Local\Temp\ipykernel_6808\445364754.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DecisionTree['cost'].loc[d] = cost
C:\Users\jay_hook\AppData\Local\Temp\ipykernel_6808\445364754.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '5.89' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  DecisionTree['cost'].loc[d] = cost
C:\Users\jay_hook\AppData\Local\Temp\ipykernel_6808\445364754.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DecisionTree['revenue'].loc[d] = revenue
C:\Us

15.48
6.79


C:\Users\jay_hook\AppData\Local\Temp\ipykernel_6808\445364754.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DecisionTree['cost'].loc[d] = np.nan
C:\Users\jay_hook\AppData\Local\Temp\ipykernel_6808\445364754.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DecisionTree['revenue'].loc[d] = np.nan
C:\Users\jay_hook\AppData\Local\Temp\ipykernel_6808\445364754.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DecisionTree['cos

In [36]:
DecisionTree['profit'] = DecisionTree['revenue'] - DecisionTree['cost']
DecisionTree

C:\Users\jay_hook\AppData\Local\Temp\ipykernel_6808\1305707979.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DecisionTree['profit'] = DecisionTree['revenue'] - DecisionTree['cost']


,DecisionTree,cost,revenue,profit
2023-10-10,0.0,5.89,NaN,NaN
2023-10-11,0.0,6.12,NaN,NaN
2023-10-12,0.0,4.92,20.03,15.11
2023-10-13,0.0,3.72,13.16,9.44
2023-10-16,0.0,3.59,13.55,9.96
2023-10-17,1.0,10.81,15.48,4.67
2023-10-18,0.0,3.25,16.32,13.07
2023-10-19,0.0,3.48,9.84,6.36
2023-10-20,1.0,11.71,6.79,-4.92
2023-10-23,0.0,NaN,NaN,NaN


SVC

In [37]:
for i in range(len(SVC.index)):
    d = SVC.index[i]
    if SVC.loc[d, 'SVC'] == 0: # 변동성 하락 양매도 스트랭글
        try:
            cost = 0
            cost += CTB9sp['S+4'].loc[d]
            cost += PTB9sp['S-4'].loc[d]
            SVC['cost'].loc[d] = cost
            revenue = 0
            revenue += CTB15sp['S+4'].loc[d]
            revenue += CTB9sp['S-4'].loc[d]
            SVC['revenue'].loc[d] = revenue
        except:
            SVC['cost'].loc[d] = np.nan
            SVC['revenue'].loc[d] = np.nan
    else: # label == 1 변동성 상승 양매수 스트랭글
        try:
            cost = 0
            cost += CTB9sp['기준행사가'].loc[d]
            cost += PTB9sp['기준행사가'].loc[d]
            SVC['cost'].loc[d] = cost
            revenue = 0
            revenue += CTB15sp['기준행사가'].loc[d]
            revenue += CTB9sp['기준행사가'].loc[d]
            SVC['revenue'].loc[d] = revenue
            print(SVC['revenue'].loc[d])
        except:
            SVC['cost'].loc[d] = np.nan
            SVC['revenue'].loc[d] = np.nan
SVC['profit'] = SVC['revenue'] - SVC['cost']
SVC

15.48
6.79


C:\Users\jay_hook\AppData\Local\Temp\ipykernel_6808\765054709.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  SVC['cost'].loc[d] = cost
C:\Users\jay_hook\AppData\Local\Temp\ipykernel_6808\765054709.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '5.89' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  SVC['cost'].loc[d] = cost
C:\Users\jay_hook\AppData\Local\Temp\ipykernel_6808\765054709.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  SVC['revenue'].loc[d] = revenue
C:\Users\jay_hook\AppData\Local\

,SVC,cost,revenue,profit
2023-10-10,0.0,5.89,NaN,NaN
2023-10-11,0.0,6.12,NaN,NaN
2023-10-12,0.0,4.92,20.03,15.11
2023-10-13,0.0,3.72,13.16,9.44
2023-10-16,0.0,3.59,13.55,9.96
2023-10-17,1.0,10.81,15.48,4.67
2023-10-18,0.0,3.25,16.32,13.07
2023-10-19,0.0,3.48,9.84,6.36
2023-10-20,1.0,11.71,6.79,-4.92
2023-10-23,1.0,NaN,NaN,NaN


RandomForest

In [38]:
for i in range(len(RandomForest.index)):
    d = RandomForest.index[i]
    if RandomForest.loc[d, 'RandomForest'] == 0: # 변동성 하락 양매도 스트랭글
        try:
            cost = 0
            cost += CTB9sp['S+4'].loc[d]
            cost += PTB9sp['S-4'].loc[d]
            RandomForest['cost'].loc[d] = cost
            revenue = 0
            revenue += CTB15sp['S+4'].loc[d]
            revenue += CTB9sp['S-4'].loc[d]
            RandomForest['revenue'].loc[d] = revenue
        except:
            RandomForest['cost'].loc[d] = np.nan
            RandomForest['revenue'].loc[d] = np.nan
    else: # label == 1 변동성 상승 양매수 스트랭글
        try:
            cost = 0
            cost += CTB9sp['기준행사가'].loc[d]
            cost += PTB9sp['기준행사가'].loc[d]
            RandomForest['cost'].loc[d] = cost
            revenue = 0
            revenue += CTB15sp['기준행사가'].loc[d]
            revenue += CTB9sp['기준행사가'].loc[d]
            RandomForest['revenue'].loc[d] = revenue
        except:
            RandomForest['cost'].loc[d] = np.nan
            RandomForest['revenue'].loc[d] = np.nan
RandomForest['profit'] = RandomForest['revenue'] - RandomForest['cost']
RandomForest

C:\Users\jay_hook\AppData\Local\Temp\ipykernel_6808\2434732125.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  RandomForest['cost'].loc[d] = cost
C:\Users\jay_hook\AppData\Local\Temp\ipykernel_6808\2434732125.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '5.89' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  RandomForest['cost'].loc[d] = cost
C:\Users\jay_hook\AppData\Local\Temp\ipykernel_6808\2434732125.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  RandomForest['revenue'].loc[d] = revenue
C:

,RandomForest,cost,revenue,profit
2023-10-10,0.0,5.89,NaN,NaN
2023-10-11,0.0,6.12,NaN,NaN
2023-10-12,0.0,4.92,20.03,15.11
2023-10-13,0.0,3.72,13.16,9.44
2023-10-16,0.0,3.59,13.55,9.96
2023-10-17,1.0,10.81,15.48,4.67
2023-10-18,0.0,3.25,16.32,13.07
2023-10-19,0.0,3.48,9.84,6.36
2023-10-20,1.0,11.71,6.79,-4.92
2023-10-23,1.0,NaN,NaN,NaN


XGB

In [39]:
for i in range(len(XGB.index)):
    d = XGB.index[i]
    if XGB.loc[d, 'XGB'] == 0: # 변동성 하락 양매도 스트랭글
        try:
            cost = 0
            cost += CTB9sp['S+4'].loc[d]
            cost += PTB9sp['S-4'].loc[d]
            XGB['cost'].loc[d] = cost
            revenue = 0
            revenue += CTB15sp['S+4'].loc[d]
            revenue += CTB9sp['S-4'].loc[d]
            XGB['revenue'].loc[d] = revenue
        except:
            XGB['cost'].loc[d] = np.nan
            XGB['revenue'].loc[d] = np.nan
    else: # label == 1 변동성 상승 양매수 스트랭글
        try:
            cost = 0
            cost += CTB9sp['기준행사가'].loc[d]
            cost += PTB9sp['기준행사가'].loc[d]
            XGB['cost'].loc[d] = cost
            revenue = 0
            revenue += CTB15sp['기준행사가'].loc[d]
            revenue += CTB9sp['기준행사가'].loc[d]
            XGB['revenue'].loc[d] = revenue
            print(XGB['revenue'].loc[d])
        except:
            XGB['cost'].loc[d] = np.nan
            XGB['revenue'].loc[d] = np.nan
XGB['profit'] = XGB['revenue'] - XGB['cost']
XGB

9.940000000000001
6.79


C:\Users\jay_hook\AppData\Local\Temp\ipykernel_6808\1862391388.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '5.89' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  XGB['cost'].loc[d] = cost
C:\Users\jay_hook\AppData\Local\Temp\ipykernel_6808\1862391388.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  XGB['cost'].loc[d] = cost
C:\Users\jay_hook\AppData\Local\Temp\ipykernel_6808\1862391388.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  XGB['revenue'].loc[d] = revenue
C:\Users\jay_hook\AppData\Loc

,XGB,cost,revenue,profit
2023-10-10,0,5.89,NaN,NaN
2023-10-11,0,6.12,NaN,NaN
2023-10-12,0,4.92,20.03,15.11
2023-10-13,0,3.72,13.16,9.44
2023-10-16,1,10.66,9.94,-0.72
2023-10-17,0,3.55,18.12,14.57
2023-10-18,0,3.25,16.32,13.07
2023-10-19,0,3.48,9.84,6.36
2023-10-20,1,11.71,6.79,-4.92
2023-10-23,1,NaN,NaN,NaN


LGBM

In [40]:
for i in range(len(LGBM.index)):
    d = LGBM.index[i]
    if LGBM.loc[d, 'LGBM'] == 0: # 변동성 하락 양매도 스트랭글
        try:
            cost = 0
            cost += CTB9sp['S+4'].loc[d]
            cost += PTB9sp['S-4'].loc[d]
            LGBM['cost'].loc[d] = cost
            revenue = 0
            revenue += CTB15sp['S+4'].loc[d]
            revenue += CTB9sp['S-4'].loc[d]
            LGBM['revenue'].loc[d] = revenue
        except:
            LGBM['cost'].loc[d] = np.nan
            LGBM['revenue'].loc[d] = np.nan
    else: # label == 1 변동성 상승 양매수 스트랭글
        try:
            cost = 0
            cost += CTB9sp['기준행사가'].loc[d]
            cost += PTB9sp['기준행사가'].loc[d]
            LGBM['cost'].loc[d] = cost
            revenue = 0
            revenue += CTB15sp['기준행사가'].loc[d]
            revenue += CTB9sp['기준행사가'].loc[d]
            LGBM['revenue'].loc[d] = revenue
        except:
            LGBM['cost'].loc[d] = np.nan
            LGBM['revenue'].loc[d] = np.nan
LGBM['profit'] = LGBM['revenue'] - LGBM['cost']
LGBM

C:\Users\jay_hook\AppData\Local\Temp\ipykernel_6808\1279832125.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  LGBM['cost'].loc[d] = cost
C:\Users\jay_hook\AppData\Local\Temp\ipykernel_6808\1279832125.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '5.89' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  LGBM['cost'].loc[d] = cost
C:\Users\jay_hook\AppData\Local\Temp\ipykernel_6808\1279832125.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  LGBM['revenue'].loc[d] = revenue
C:\Users\jay_hook\AppData\

,LGBM,cost,revenue,profit
2023-10-10,0.0,5.89,NaN,NaN
2023-10-11,0.0,6.12,NaN,NaN
2023-10-12,0.0,4.92,20.03,15.11
2023-10-13,0.0,3.72,13.16,9.44
2023-10-16,1.0,10.66,9.94,-0.72
2023-10-17,0.0,3.55,18.12,14.57
2023-10-18,0.0,3.25,16.32,13.07
2023-10-19,0.0,3.48,9.84,6.36
2023-10-20,1.0,11.71,6.79,-4.92
2023-10-23,1.0,NaN,NaN,NaN
